Here I only test parts of the code before passing them on to the main notebook.

In [1]:
import time
import requests
from dotenv import load_dotenv
import os
import folium
import pandas as pd
import json

In [2]:
m = folium.Map(location=[20.9939879883004, -89.62853393602846], zoom_start=12)
delta = 0.01

initial_lat, initial_lng = 20.944451,-89.654108 


rectangles_viewports = []

for i in range(6):
    for j in range(6):
        low = [initial_lat+i*delta, initial_lng+j*delta]
        high = [initial_lat+(i+1)*delta, initial_lng+(j+1)*delta]
        folium.Rectangle(
            bounds = [low, high],
            tooltip = f'({i+1},{j+1})',
            fill = True
        ).add_to(m)
        
        rectangles_viewports.append((tuple(low), tuple(high)))
m

In [3]:
print(rectangles_viewports)
print(len(rectangles_viewports))

[((20.944451, -89.654108), (20.954451000000002, -89.64410799999999)), ((20.944451, -89.64410799999999), (20.954451000000002, -89.634108)), ((20.944451, -89.634108), (20.954451000000002, -89.62410799999999)), ((20.944451, -89.62410799999999), (20.954451000000002, -89.61410799999999)), ((20.944451, -89.61410799999999), (20.954451000000002, -89.604108)), ((20.944451, -89.604108), (20.954451000000002, -89.59410799999999)), ((20.954451000000002, -89.654108), (20.964451, -89.64410799999999)), ((20.954451000000002, -89.64410799999999), (20.964451, -89.634108)), ((20.954451000000002, -89.634108), (20.964451, -89.62410799999999)), ((20.954451000000002, -89.62410799999999), (20.964451, -89.61410799999999)), ((20.954451000000002, -89.61410799999999), (20.964451, -89.604108)), ((20.954451000000002, -89.604108), (20.964451, -89.59410799999999)), ((20.964451, -89.654108), (20.974451000000002, -89.64410799999999)), ((20.964451, -89.64410799999999), (20.974451000000002, -89.634108)), ((20.964451, -89.

In [4]:
load_dotenv()
API_KEY = os.getenv('API_KEY')

search_url = "https://places.googleapis.com/v1/places:searchText"
headers = {
    'Content-Type' : 'application/json',
    'X-Goog-Api-Key': API_KEY,
    'X-Goog-FieldMask': 'places.id,nextPageToken'
}

In [ ]:
raw_data = []
for low, high in rectangles_viewports:
    try: 
        payload = {
            'textQuery' : 'cafeteria',
            'includedType': 'cafe',
            'strictTypeFiltering':True,
            'pageSize' : 20,
            'locationRestriction' : {
                'rectangle':{
                    'low':{
                        'latitude' : low[0],
                        'longitude' : low[1]
                    },
                    'high':{
                        'latitude': high[0],
                        'longitude': high[1]
                        
                    }
                }
            }
        }
        
        response = requests.post(url = search_url, json=payload, headers = headers)
        time.sleep(0.5)
        response.raise_for_status()
        
        data = response.json()
        print(data)
        
        raw_data.extend(data.get('places', []))
        
    except requests.exceptions.RequestException as e:
        print(f'ERROR!!! --> {e}')

with open('test_ids.json', 'w') as f:
    json.dump(raw_data, f, indent=4)

In [5]:
with open('test_ids.json', 'r') as file:
    ids_dict = json.load(file)

len(ids_dict)

178

In [10]:
details_headers = {
    'Content-Type' : 'application/json',
    'X-Goog-Api-Key': API_KEY,
    'X-Goog-FieldMask':'displayName,formattedAddress,location,businessStatus,primaryTypeDisplayName,priceRange,rating,userRatingCount,websiteUri'
}

In [11]:
detail_raw_data = []

for place in ids_dict:
    try:
        details_url = f'https://places.googleapis.com/v1/places/{place['id']}'
        
        response = requests.get(url=details_url, headers=details_headers)
        time.sleep(0.5)
        response.raise_for_status()
        
        data = response.json()
        
        
        print(data)
        detail_raw_data.append(data)
        
        
    except requests.exceptions.RequestException as e:
        print(f'ERROR!!! --> {e}')

pd.DataFrame(detail_raw_data).to_csv('test_df.csv')

{'formattedAddress': 'Aviación 709, Nueva Sambulá, 97250 Mérida, Yuc., Mexico', 'location': {'latitude': 20.9481689, 'longitude': -89.6512338}, 'rating': 4.4, 'websiteUri': 'http://www.italiancoffee.com/', 'businessStatus': 'OPERATIONAL', 'userRatingCount': 809, 'displayName': {'text': 'The Italian Coffee', 'languageCode': 'en'}, 'primaryTypeDisplayName': {'text': 'Cafe', 'languageCode': 'en-US'}, 'priceRange': {'startPrice': {'currencyCode': 'MXN', 'units': '100'}, 'endPrice': {'currencyCode': 'MXN', 'units': '200'}}}
{'formattedAddress': 'C. 79ᴮ 534, Itzaes, 97259 Mérida, Yuc., Mexico', 'location': {'latitude': 20.954439400000002, 'longitude': -89.6472172}, 'rating': 4.7, 'businessStatus': 'OPERATIONAL', 'userRatingCount': 3, 'displayName': {'text': 'Casa de Pedrito', 'languageCode': 'es'}, 'primaryTypeDisplayName': {'text': 'Coffee Shop', 'languageCode': 'en-US'}}
{'formattedAddress': 'Nueva Sambulá, 97250 Mérida, Yuc., Mexico', 'location': {'latitude': 20.950837399999998, 'longitud